## Workshop - Regularization

In this workshop, we are going to:

1. Tune an elastic-net regression 
2. Compare the following models:
    1. The null model
    2. The tuned elastic-net model
    3. The trimmed non-regularized model with standardized features
    4. The trimmed non-regularized model with non-standardized features
    
# Preliminaries

- Load any necessary packages and/or functions
- Load in and prepare the class data
- Create x and y with a label of `pct_d_rgdp`
- Create `x_train`, `x_test`, `y_train`, `y_test` with
    * training size of two-thirds
    * random state of 490
- Standardize the features
- Add constants

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import linear_model as lm

In [4]:
df = pd.read_pickle('C:/Users/Chris/Documents/Spring 2021/Module 2 Stuff/class_data.pkl')
df.columns

df_prepped = df.drop(columns = ['urate_bin', 'year']).join([
    pd.get_dummies(df['urate_bin'], drop_first = True),
    pd.get_dummies(df.year, drop_first = True)    
])

In [5]:
y = df_prepped['pct_d_rgdp']
x = df_prepped.drop(columns = 'pct_d_rgdp')

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size = 2/3, random_state = 490)

In [7]:
x_train_std = x_train.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)
x_test_std  = x_test.apply(lambda x: (x - np.mean(x))/np.std(x), axis = 0)

x_train_std = sm.add_constant(x_train_std)
x_test_std  = sm.add_constant(x_test_std)
x_train     = sm.add_constant(x_train)
x_test      = sm.add_constant(x_test)

Take a look at `lm.ElasticNet?` and 
```
fit = sm.OLS(y_train, x_train)
fit.fit_regularized?
```
Determine which coefficients are the same, but named differently.
Specifically, $\alpha$ and the weight on the different constraints (i.e. $||\beta||_2$ and $||\beta||_1$).

In [38]:
lm.ElasticNet?
#lm_fit = lm.LinearRegression().fit()

#alpha = 1, l1_ratio =  .5

#Unsure how to enter in the arguements :/
#lm_elastic = lm_fit.ElasticNet(alpha = 1, x_train_std, y_train)

Init signature:
lm.ElasticNet(
    alpha=1.0,
    *,
    l1_ratio=0.5,
    fit_intercept=True,
    normalize=False,
    precompute=False,
    max_iter=1000,
    copy_X=True,
    tol=0.0001,
    warm_start=False,
    positive=False,
    random_state=None,
    selection='cyclic',
)
Docstring:     
Linear regression with combined L1 and L2 priors as regularizer.

Minimizes the objective function::

        1 / (2 * n_samples) * ||y - Xw||^2_2
        + alpha * l1_ratio * ||w||_1
        + 0.5 * alpha * (1 - l1_ratio) * ||w||^2_2

If you are interested in controlling the L1 and L2 penalty
separately, keep in mind that this is equivalent to::

        a * L1 + b * L2

where::

        alpha = a + b and l1_ratio = a / (a + b)

The parameter l1_ratio corresponds to alpha in the glmnet R package while
alpha corresponds to the lambda parameter in glmnet. Specifically, l1_ratio
= 1 is the lasso penalty. Currently, l1_ratio <= 0.01 is not reliable,
unless you supply your own sequence of alpha.

R

In [39]:
fit = sm.OLS(y_train, x_train)
fit.fit_regularized?

Signature:
fit.fit_regularized(
    method='elastic_net',
    alpha=0.0,
    L1_wt=1.0,
    start_params=None,
    profile_scale=False,
    refit=False,
    **kwargs,
)
Docstring:
Return a regularized fit to a linear regression model.

Parameters
----------
method : str
    Either 'elastic_net' or 'sqrt_lasso'.
alpha : scalar or array_like
    The penalty weight.  If a scalar, the same penalty weight
    applies to all variables in the model.  If a vector, it
    must have the same length as `params`, and contains a
    penalty weight for each coefficient.
L1_wt : scalar
    The fraction of the penalty given to the L1 penalty term.
    Must be between 0 and 1 (inclusive).  If 0, the fit is a
    ridge fit, if 1 it is a lasso fit.
start_params : array_like
    Starting values for ``params``.
profile_scale : bool
    If True the penalized fit is computed using the profile
    (concentrated) log-likelihood for the Gaussian model.
    Otherwise the fit uses the residual sum of squares.
ref

const                0.0
pos_net_jobs         0.0
emp_estabs           0.0
estabs_entry_rate    0.0
estabs_exit_rate     0.0
pop                  0.0
pop_pct_black        0.0
pop_pct_hisp         0.0
lfpr                 0.0
density              0.0
lower                0.0
similar              0.0
2003                 0.0
2004                 0.0
2005                 0.0
2006                 0.0
2007                 0.0
2008                 0.0
2009                 0.0
2010                 0.0
2011                 0.0
2012                 0.0
2013                 0.0
2014                 0.0
2015                 0.0
2016                 0.0
2017                 0.0
2018                 0.0
dtype: float64

Perform a 5-fold cross-validation grid search with a random state of 490. 
Identify the optimally tuned hyperparameters.
Use this grid:
```
param_grid = {'alpha': 10.**np.arange(-5, -1, 1), 
              'l1_ratio': np.arange(0, 1, 0.1)}
```
You will get a warning message about convergence.
We will discuss it after the workshop.
Think about why it occuring.

****
# Question

How many models did we just fit?

***
Using the tuned hyperparameters, fit your elastic net model with `statsmodels`

Using the selected features refit

- the non-regularized model with standardized features
- the non-regularized model with non-standardized features

Compare the percent improvement from the null model RMSE to the elastic-net and OLS model.